<a href="https://colab.research.google.com/github/Chirag314/QA-chatbot/blob/main/ADaM_QAbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial we will build PDF QA bot using open source models. We will show how commercially available open source models to query the knowledgebase.

  - This notebook has been created using using following reference.https://github.com/devsentient/examples/blob/main/LLMs/QA_app/pdf_qa.ipynb

In [ ]:
#Installations
'''
!pip install langchain==0.0.189
!pip install chromadb==0.3.25
!pip install pdfplumber==0.9.0
!pip install tiktoken==0.4.0
!pip install lxml==4.9.2
!pip install torch==2.0.1

!pip install accelerate==0.19.0
!pip install sentence-transformers==2.2.2
!pip install einops==0.6.1
!pip install xformers==0.0.20
'''

In [2]:
#Installations
!pip install langchain
!pip install chromadb
!pip install pdfplumber
!pip install tiktoken
!pip install lxml
!pip install torch

!pip install accelerate
!pip install sentence-transformers
!pip install einops
!pip install xformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.6/508.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
!pip install transformers
!pip install tensorflow
from tensorflow import keras
#!pip instll keras

OpenAI keys

In [28]:
#!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [4]:
from dotenv import load_dotenv
import json
import os
try:
  from hyperplane.utils import is_jhub
  if is_jhub():
    openaiKeyFile='/root/.secret/openai_key.json'
  else:
    openaiKeyFile='/etc/hyperplane/secrets/openai_key.json'
  with open(openaiKeyFile) as f:
    os.environ['OPENAI_API_KEY']=json.load(f)['openai_key']
except Exception as e:
  print(e)
  load_dotenv()

No module named 'hyperplane'


##Step 1: Loading LLM embedding models and generative models

### LLM EMbedding Models
- opensource models used are
  -INSTRUCTOR XL:Instructor xl is an instruction-finetuned text embedding model, that can generate embeddings tailored for any task instuction. The instruction for embedding text snippets is "Represent the document for retrieval:" and instruction for embedding user question is "Represent the question for retrieving supporting documents:"

  -SBERT:SBERT maps sentences and paragraphs to a vectore using BERT like model. It's a good starter when you're prototyping your application

### LLM Generation Models
  - FlanT5 Models: FlanT5 is text2text generator that is finetuned on a number of tasks like summarisation, question answering. It uses the encode-decoder architecture of transformers. The model is Apache 2.0 licensed. It can used commercially.

  - FastChatT5 3b Model: It's a FlanT5 based chat model trained by finetuning FlanT5 on user chats from ChatGpt. The model is Apache 2.0 licensed.

  - Falcon7b Model: Falcon7b is a smaller version of Falcon which is text generator model (decoder only model). Falcon-40B is currently the best open source model on the OpenLLM Leaderboard. They are trained with high quality training data

In [5]:
EMB_OPENAI_ADA = "text-embedding-ada-002"
EMB_INSTRUCTOR_XL = "hkunlp/instructor-xl"
EMB_SBERT_MPNET_BASE = "sentence-transformers/all-mpnet-base-v2"

In [6]:
LLM_OPENAI_GPT35="gpt-3.5-turbo"
LLM_FLAN_T5_XXL="google/flan-t5-xxl"
LLM_FLAN_T5_XL='google/flan-t5-xl'
LLM_FASTCHAT_T5_XL = "lmsys/fastchat-t5-3b-v1.0"
LLM_FLAN_T5_SMALL = "google/flan-t5-small"
LLM_FLAN_T5_BASE = "google/flan-t5-base"
LLM_FLAN_T5_LARGE = "google/flan-t5-large"
LLM_FALCON_SMALL = "tiiuae/falcon-7b-instruct"


ERROR: Could not find a version that satisfies the requirement tensorflo==2.14.0 (from versions: none)
ERROR: No matching distribution found for tensorflo==2.14.0


Lets go ahead and first setup SBERT for embedding model and flant5-base for generation model.

In [10]:
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
import torch
from transformers import AutoTokenizer
import re
#import keras

In [11]:
config={"persist_directory":None,
        "load_in_8bit":False,
        "embedding":EMB_SBERT_MPNET_BASE,
        "llm":LLM_FLAN_T5_BASE,
        }

We can use huggingface pipelines to load model which makes our job easy. We will create few helper functions here.

In [12]:
def create_sbert_mpnet():
  device="cuda" if torch.cuda.is_available() else "cpu"
  return HuggingFaceEmbeddings(model_name=EMB_SBERT_MPNET_BASE,model_kwargs={"device": device})

def create_flan_t5_base(load_in_8bit=False):
  model='google/flan-t5-base'
  tokenizer=AutoTokenizer.from_pretrained(model)
  return pipeline(
         task='text2text-generation',
         model=model,
         tokenizer=tokenizer,
         max_new_tokens=100,
         model_kwargs={"device_map": "auto", "load_in_8bit": load_in_8bit, "max_length": 512, "temperature": 0.}
        )

def create_falcon_instruct_small(load_in_8bit=False):
  model='tiiuae/falcon-7b-instruct'
  tokenizer=AutoTokenizer.from_pretrained(model)
  hf_pipeline=pipeline(
      task='text_generation',
      model=model,
      tokenizer=tokenizer,
      trust_remote_code=True,
      max_new_tokens=100,
      model_kwargs={
          "device_map":"auto",
          "load_in_8bit":load_in_8bit,
          "max_length":512,
          "temperature":0.01,
          "torch_dtype":torch.bfloat16,
      }
  )

In [13]:
if config['embedding']==EMB_SBERT_MPNET_BASE:
  embedding=create_sbert_mpnet()


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [16]:
!pip install --upgrade tensorflow
!pip install  transformer
#from tensorflow import keras
load_in_8bit=config['load_in_8bit']
if config['llm']==LLM_FLAN_T5_BASE:
  llm=create_flan_t5_base(load_in_8bit=load_in_8bit)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.2 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


RuntimeError: ignored